In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time
import csv
import pandas as pd

In [65]:
url = 'https://www.marmiton.org/recettes/recherche.aspx?aqt=sandwich&start=96&page=9'


npo_recipe = []

    
response = requests.get(url)
data = response.text
soup = BeautifulSoup(data,'html.parser')
recipes = soup.find_all('div',{'class':'recipe-card'})
    
for recipe in recipes:
    
    npo_recipe.append({'title':recipe.find('h4',{'class':"recipe-card__title"}).text,
    'description':recipe.find('div',{'class': 'recipe-card__description'}).text,
    'duration':recipe.find('span',{'class': 'recipe-card__duration__value'}).text,
    'url' : recipe.find('a',{'class': 'recipe-card-link'}).get('href'),
    'url1' : ('https://www.marmiton.org/'+ url)}) 


  



In [66]:
npo_recipe

[{'title': 'Sandwichs sucrés',
  'description': '\nIngrédients : quatre-quarts au beurre Ker Cadélac,confiture de fraise,pâte à tartiner,chocolat noir fondu,noisettes,myrtilles,poire,menthe.Sandwich sucré aux baies: Coupez 2 tranches de quatre-quarts.Tartinez le...\t\t',
  'duration': '30 min',
  'url': '/recettes/recette_sandwichs-sucres_166017.aspx',
  'url1': 'https://www.marmiton.org/https://www.marmiton.org/recettes/recherche.aspx?aqt=sandwich&start=96&page=9'},
 {'title': 'Sandwiches glacés vanille-framboise et salade de fruits rouges',
  'description': '\nIngrédients : fraises coupées en tranches épaisses,sucre en poudre,biscuit sec rectangulaires de votre choix,framboises,glace vanille,framboises.Mélanger les fraises et le sucre dans un saladier.Former les...\t\t',
  'duration': '10 min',
  'url': '/recettes/recette_sandwichs-glaces-vanille-frambroise-et-salade-de-fruits-rouges_312633.aspx',
  'url1': 'https://www.marmiton.org/https://www.marmiton.org/recettes/recherche.aspx?aq

In [67]:
df9 = pd.DataFrame(npo_recipe) 

In [2]:
df = pd.read_csv('SandwichMarmiton.csv')

In [3]:
df.head()

,Unnamed: 0,description,duration,title,url,url1
0,0,"\nIngrédients : pain de mie,escalope de poulet...",50 min,Sandwichs américains,/recettes/recette_sandwichs-americains_34637.aspx,https://www.marmiton.org/https://www.marmiton....
1,1,\nIngrédients : pain de mie anglais (le super ...,5 min,Club sandwich poulet - bacon,/recettes/recette_club-sandwich-poulet-bacon_4...,https://www.marmiton.org/https://www.marmiton....
2,2,\nIngrédients : escalope de poulet coupées en ...,25 min,Poulet mariné en sandwich frais au pain pita,/recettes/recette_poulet-marine-en-sandwich-fr...,https://www.marmiton.org/https://www.marmiton....
3,3,"\nIngrédients : pain de mie complet,concombre ...",10 min,Sandwichs de concombre,/recettes/recette_sandwichs-de-concombre_63337...,https://www.marmiton.org/https://www.marmiton....
4,4,\nIngrédients : Émietté Végétal citron & poivr...,20 min,Club sandwich au coleslaw et à l’Emietté Végét...,/recettes/recette_club-sandwich-au-coleslaw-et...,https://www.marmiton.org/https://www.marmiton....


In [4]:
df['description'][100]

'\nIngrédients : pain complet américain (extra moelleux),cheddar,blanc de poulet rôti (jambon de poulet),beurre salé (facultatif),salade iceberg.Beurrer légèrement les tranches de pain.Mettre une tranche de cheddar...\t\t'

In [5]:
df.shape

(135, 6)

In [6]:
df.isnull().sum()

Unnamed: 0     0
description    0
duration       2
title          0
url            0
url1           0
dtype: int64

In [15]:
from pandas_profiling import ProfileReport

ProfileReport(df)

In [7]:
# engineer features

def engineering_features(X):
    
    X = X.copy()
    
#     X['title'] = X['title'].str.contains('sandwich')
    
    X = X.drop(columns=['url'])
    
    return X
    

In [8]:
df = engineering_features(df)

In [9]:
df.head()

,Unnamed: 0,description,duration,title,url1
0,0,"\nIngrédients : pain de mie,escalope de poulet...",50 min,Sandwichs américains,https://www.marmiton.org/https://www.marmiton....
1,1,\nIngrédients : pain de mie anglais (le super ...,5 min,Club sandwich poulet - bacon,https://www.marmiton.org/https://www.marmiton....
2,2,\nIngrédients : escalope de poulet coupées en ...,25 min,Poulet mariné en sandwich frais au pain pita,https://www.marmiton.org/https://www.marmiton....
3,3,"\nIngrédients : pain de mie complet,concombre ...",10 min,Sandwichs de concombre,https://www.marmiton.org/https://www.marmiton....
4,4,\nIngrédients : Émietté Végétal citron & poivr...,20 min,Club sandwich au coleslaw et à l’Emietté Végét...,https://www.marmiton.org/https://www.marmiton....


In [10]:
df.shape

(135, 5)

In [11]:
df['description'][13]

"\nIngrédients : pain de mie sans croûte,fromage frais type Saint Morêt,saumon fumé,aneth,citron,salade sucrine,poivre.A l'aide d'un cercle à pâtisserie, détailler les tranches de pain de mie en ronds. Hacher...\t\t"

In [12]:
# vectorize the description (transformer les mot en chiffre)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

dtm  = tfidf.fit_transform(df['description']) 

dtm = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())

dtm.head()








,10,100,12,14,150,180,180c,1ère,20,200,...,épinards,éplucher,épépiner,épépinées,équeuter,équivalent,étaler,étalez,étapes,être
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.224218,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.617882,0.0,0.0,0.0,0.0


In [13]:
# query Description by similarity 

from sklearn.neighbors import NearestNeighbors

# instantiate model

nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')


# fit the model

nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [14]:
# Query using kneighbors
nn.kneighbors([dtm.iloc[0]])

(array([[0.        , 1.22127997, 1.2444048 , 1.24806432, 1.25018773]]),
 array([[ 0, 68, 11, 46, 87]]))

In [15]:
df['description'][0]

'\nIngrédients : pain de mie,escalope de poulet ou de rôti de porc cuit,jambon,tomate,salade,mayonnaise (quantité selon le goût),beurre à tartiner, environ 30 g,moutarde  environ 1,5 cuillère à café.Couper les...\t\t'

In [16]:
import pickle 

filename = 'vect_01.pkl'
pickle.dump(tfidf, open(filename, 'wb'))

filename = 'knn_01.pkl'
pickle.dump(nn, open(filename, 'wb'))